In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dropout, Dense, MaxPool2D
from tensorflow.keras.layers import Input, LeakyReLU, Softmax, Reshape, Flatten
from tensorflow.keras.layers import concatenate, maximum, Lambda, Layer
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
import tensorflow.keras.backend as K
import pickle

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Additional layers not available in core Keras

In [2]:
class Bias(Layer):
    """
    Adds bias to a layer. This is used for untied biases convolution. 
    """
    def __init__(self, **kwargs):
        super(Bias, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.bias = self.add_weight(name='bias', 
                                      shape=input_shape[1:],
                                      initializer='uniform',
                                      trainable=True)
        super(Bias, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return tf.add(x, self.bias)

    def compute_output_shape(self, input_shape):
        return input_shape

In [3]:
def feature_pool_max(input, pool_size=2, axis=1):
    """
    Based on lasagne implementation of FeaturePool
    """
    input_shape = input.shape.as_list()
    num_feature_maps = input_shape[axis]
    num_feature_maps_out = num_feature_maps // pool_size
    
    pool_shape = tf.TensorShape(
        (input_shape[1:axis] + [num_feature_maps_out, pool_size] + input_shape[axis+1:])
    )
    
    print(pool_shape)
    input_reshaped = Reshape(pool_shape)(input)
    # reduce along all axis but the target one
    reduction_axis = list(range(1, len(pool_shape)+1))
    reduction_axis.pop(axis-1)
    
    return tf.reduce_max(input_reshaped, axis=reduction_axis)

# create Layer for reshape with batchsize
# strong_reshape_func = lambda x: tf.reshape(x, (batch_size//2, concat.shape[1]*2))
# StrongReshape = Lambda(strong_reshape_func)

# Building the main model

In [20]:
def build_model(width=512, height=512, filename=None,
                n_classes=5, batch_size=64, p_conv=0.0):
    """
    
    """
    # Input shape (height, width, depth), different from original implimentation
    main_input = Input(shape=(height, width, 3),
                       batch_size=batch_size,
                      )
    
    # Note: for conv layers paper uses untie_biases=True
    # layer will have separate bias parameters for each position in each channel. 
    # As a result, the b attribute will be a 3D tensor.

    # no need to init weights as they will be loaded from a file
    # Conv layers(filters, kernel_size)
    conv_main_1 = Conv2D(32, 7, strides=(2, 2), padding='same', 
                    use_bias=False,
                    activation= None,
                   )(main_input)
    conv_bias_1 = Bias()(conv_main_1)
    conv_activation_1 = LeakyReLU(alpha=0.5)(conv_bias_1)
    dropout_1 = Dropout(p_conv)(conv_activation_1)
    maxpool_1 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_1)
    # 3
    conv_main_2 = Conv2D(32, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation= None,
                        )(maxpool_1)
    conv_bias_2 = Bias()(conv_main_2)
    conv_activation_2 = LeakyReLU(alpha=0.5)(conv_bias_2)
    dropout_2 = Dropout(p_conv)(conv_activation_2)
    # 4
    conv_main_3 = Conv2D(32, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation= None,
                        )(dropout_2)
    conv_bias_3 = Bias()(conv_main_3)
    conv_activation_3 = LeakyReLU(alpha=0.5)(conv_bias_3)
    dropout_3 = Dropout(p_conv)(conv_activation_3)
    maxpool_3 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_3)
    # 6
    conv_main_4 = Conv2D(64, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(maxpool_3)
    conv_bias_4 = Bias()(conv_main_4)
    conv_activation_4 = LeakyReLU(alpha=0.5)(conv_bias_4)
    dropout_4 = Dropout(p_conv)(conv_activation_4)
    # 7
    conv_main_5 = Conv2D(64, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_4)
    conv_bias_5 = Bias()(conv_main_5)
    conv_activation_5 = LeakyReLU(alpha=0.5)(conv_bias_5)
    dropout_5 = Dropout(p_conv)(conv_activation_5)
    maxpool_5 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_5)
    # 9 
    conv_main_6 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(maxpool_5)
    conv_bias_6 = Bias()(conv_main_6)
    conv_activation_6 = LeakyReLU(alpha=0.5)(conv_bias_6)
    dropout_6 = Dropout(p_conv)(conv_activation_6)
    # 10    
    conv_main_7 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_6)
    conv_bias_7 = Bias()(conv_main_7)
    conv_activation_7 = LeakyReLU(alpha=0.5)(conv_bias_7)
    dropout_7 = Dropout(p_conv)(conv_activation_7)
    # 11
    conv_main_8 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_7)
    conv_bias_8 = Bias()(conv_main_8)
    conv_activation_8 = LeakyReLU(alpha=0.5)(conv_bias_8)
    dropout_8 = Dropout(p_conv)(conv_activation_8)
    # 12
    conv_main_9 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_8)
    conv_bias_9 = Bias()(conv_main_9)
    conv_activation_9 = LeakyReLU(alpha=0.5)(conv_bias_9)
    dropout_9 = Dropout(p_conv)(conv_activation_9)
    maxpool_9 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_9)
    # 14
    conv_main_10 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(maxpool_9)
    conv_bias_10 = Bias()(conv_main_10)
    conv_activation_10 = LeakyReLU(alpha=0.5)(conv_bias_10)
    dropout_10 = Dropout(p_conv)(conv_activation_10)
    # 15
    conv_main_11 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_10)
    conv_bias_11 = Bias()(conv_main_11)
    conv_activation_11 = LeakyReLU(alpha=0.5)(conv_bias_11)
    dropout_11 = Dropout(p_conv)(conv_activation_11)
    # 16
    conv_main_12 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_11)
    conv_bias_12 = Bias()(conv_main_12)
    conv_activation_12 = LeakyReLU(alpha=0.5)(conv_bias_12)
    dropout_12 = Dropout(p_conv)(conv_activation_12)
    # 17
    conv_main_13 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_12)
    conv_bias_13 = Bias()(conv_main_13)
    conv_activation_13 = LeakyReLU(alpha=0.5, name='layer_17')(conv_bias_13)
    dropout_13 = Dropout(p_conv)(conv_activation_13)
    maxpool_13 = MaxPool2D(pool_size=3, name="last_conv", strides=(2, 2))(dropout_13)
    # 19, special dropout between phases with p=1/2 
    dropout_inter = Dropout(0.5)(maxpool_13)
    # 20 Dense phase 
    # Begins with Maxout layer, which is implemented here as Dense+custom feature_pool function
    flatten_inter = Flatten()(dropout_inter)
    maxout_1 = Dense(units=1024, 
                     activation=None)(flatten_inter)
    # need to wrap operation in Lambda to count as a layer
    maxout_2 = Lambda(lambda x: feature_pool_max(x, pool_size=2, axis=1))(maxout_1)
    
    # 22 Concatenate with processed img, take both eyes into account
    img_dim_input = Input(shape=(2,), batch_size=batch_size, name="imgdim")
    concat = concatenate([maxout_2, img_dim_input], axis=1)
    
    # 24
    # flatten = Reshape((-1, concat.output_shape()[1] * 2))(concat)
    print(concat.shape)
    # use lambda for custom reshape that's capable of changing batch_size as well
    # expect order left-right
    # TODO: (-1, net['23'].output_shape[1] * 2)
    flatten = Lambda(lambda x: K.reshape(x, (-1, concat.shape[1]*2)))(concat)
    dense_droupout_0 = Dropout(0.5)(flatten)
    # 26
    dense_1 = Dense(units=1024,
                    activation=None,
                   )(dense_droupout_0)
    dense_maxpool_1 = Lambda(lambda x: feature_pool_max(x, pool_size=2, axis=1))(dense_1)
    dense_dropout_1 = Dropout(0.5)(dense_maxpool_1)
    
    # 29
    dense_2 = Dense(units=n_classes*2,
                    activation=None,
                   )(dense_dropout_1)
    softmax_flatten = Lambda(lambda x: tf.reshape(x, (-1, n_classes)))(dense_2)
    softmax = keras.layers.Softmax()(softmax_flatten)
    
    model = Model(inputs=[main_input, img_dim_input], outputs=[softmax])
    
    return model

In [21]:
model = build_model()
model.summary()

(512, 2)
(64, 514)
(512, 2)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (64, 512, 512, 3)    0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (64, 256, 256, 32)   4704        input_3[0][0]                    
__________________________________________________________________________________________________
bias_26 (Bias)                  (64, 256, 256, 32)   2097152     conv2d_26[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_26 (LeakyReLU)      (64, 256, 256, 32)   0           bias_26[0][0]                    
_________________________________________________________________________________

## Loading weights

Due to unicode compatability issues between python2 and python3 we use np.save method, with encoding as 'bytes'.

In [22]:
params_names = ['Conv_1_W', 'Conv_1_b', 'Conv_2_W', 'Conv_2_b', 'Conv_3_W', 'Conv_3_b', 
                'Conv_4_W', 'Conv_4_b', 'Conv_5_W', 'Conv_5_b', 'Conv_6_W', 'Conv_6_b', 
                'Conv_7_W', 'Conv_7_b', 'Conv_8_W', 'Conv_8_b', 'Conv_9_W', 'Conv_9_b', 
                'Conv_10_W', 'Conv_10_b', 'Conv_11_W', 'Conv_11_b', 'Conv_12_W', 'Conv_12_b', 
                'Conv_13_W', 'Conv_13_b', 'Dense_1_W', 'Dense_1_b', 'Dense_2_W', 'Dense_2_b', 
                'Dense_3_W', 'Dense_3_b']

In [23]:
arr = np.load("../models/JFnet_weights.npy", encoding="bytes")

In [24]:
layer_to_weights = dict(zip(params_names, arr))
layer_to_weights['Dense_3_b']

array([ 0.5916246 ,  0.04647866,  0.40189323, -0.12301593, -0.41697878,
        0.6376505 ,  0.02411902,  0.393416  , -0.03656115, -0.51861304],
      dtype=float32)

In [41]:
def update_weights(model, layer_to_weights):
    # set all layers in the model
    c_conv = 1
    c_dense = 1
    # lasagne store weights not their transpose, so we transpose
    for layer in model.layers:
        layer_str = str(layer)
        if "Conv2D" in layer_str:
            layer.set_weights([
                layer_to_weights['Conv_{}_W'.format(c_conv)].T
            ])
        if "Bias" in layer_str:
            layer.set_weights([
                layer_to_weights['Conv_{}_b'.format(c_conv)].T
            ])
            c_conv +=1
        if "Dense" in layer_str:
            layer.set_weights([
                layer_to_weights['Dense_{}_W'.format(c_dense)],
                layer_to_weights['Dense_{}_b'.format(c_dense)]
            ])
            c_dense += 1

update_weights(model, layer_to_weigts)

ValueError: Layer weight shape (512, 2) not compatible with provided weight shape (12544, 1024)

# Save models with these weights for future use in the class

In [26]:
model.save_weights("../models/keras_JFnet.h5")

In [27]:
model2 = build_model()

(512, 2)
(64, 514)
(512, 2)


In [28]:
model2.input_shape

[(64, 512, 512, 3), (64, 2)]

In [29]:
model.predict([np.zeros((64, 512, 512, 3)), np.zeros((64, 2))])

array([[0.29261371, 0.26478586, 0.3295034 , 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923905],
       [0.29261371, 0.26478586, 0.3295034 , 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923905],
       [0.29261371, 0.26478586, 0.3295034 , 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923905],
       [0.29261371, 0.26478586, 0.3295034 , 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923905],
       [0.29261371, 0.26478586, 0.3295034 , 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923905],
       [0.29261371, 0.26478586, 0.3295034 , 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923905],
       [0.29261371, 0.26478586, 0.3295034 , 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923905],
       [0.29261371, 0.26478586, 0.

In [30]:
model2.load_weights("../models/keras_JFnet.h5")

## Peeling off layers and BCNN

In [43]:
n_classes=2 

In [44]:
conv_output = model2.get_layer("layer_17").output

mean_pooled = GlobalAveragePooling2D(
    data_format='channels_last')(conv_output)
max_pooled = GlobalMaxPooling2D(
    data_format='channels_last')(conv_output)
global_pool = concatenate([mean_pooled, max_pooled], axis=1)

softmax_input = Dense(
    units=n_classes, activation=None,)(global_pool)
softmax_output = Softmax()(softmax_input)

bcnn_model = Model(inputs=[model2.input[0]], outputs=[softmax_output])

In [45]:
bcnn_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (64, 512, 512, 3)    0                                            
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (64, 256, 256, 32)   4704        input_4[0][0]                    
__________________________________________________________________________________________________
bias_39 (Bias)                  (64, 256, 256, 32)   2097152     conv2d_39[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_38 (LeakyReLU)      (64, 256, 256, 32)   0           bias_39[0][0]                    
__________________________________________________________________________________________________
dropout_48

## Saving bcnn models

In [60]:
bcnn_param_names = ['Conv_1_W', 'Conv_1_b', 'Conv_2_W', 'Conv_2_b', 'Conv_3_W', 'Conv_3_b', 
                    'Conv_4_W', 'Conv_4_b', 'Conv_5_W', 'Conv_5_b', 'Conv_6_W', 'Conv_6_b', 
                    'Conv_7_W', 'Conv_7_b', 'Conv_8_W', 'Conv_8_b', 'Conv_9_W', 'Conv_9_b', 
                    'Conv_10_W', 'Conv_10_b', 'Conv_11_W', 'Conv_11_b', 'Conv_12_W', 'Conv_12_b', 
                    'Conv_13_W', 'Conv_13_b', 'Dense_1_W', 'Dense_1_b']

In [67]:
bcnn_weights = np.load("../models/weights_bcnn1_392bea6.npy", encoding="bytes")

In [68]:
len(bcnn_weights) == len(params_names)

True

In [69]:
layer_to_weights = dict(zip(params_names, bcnn_weights))

In [70]:
update_weights(bcnn_model, layer_to_weights)

In [71]:
bcnn_model.save_weights("../models/weights_bcnn1_392bea6.h5")

## Prediction with uncertainty

In [ ]:
model_input = np.zeros(model.input_shape)
model2_input = [np.zeros((64, 512, 512, 3)), np.zeros((64, 2))]

In [18]:
# prediction with uncertainty
f = K.function(
    [model2.input, K.learning_phase()],
    model2.layers[-1].output,
)

In [19]:
f([model2_input, 1])

array([[0.39036417, 0.2853371 , 0.23163301, 0.0724251 , 0.02024073],
       [0.34003672, 0.26134565, 0.29670554, 0.08337192, 0.01854014],
       [0.3549058 , 0.29158244, 0.28603002, 0.05870101, 0.00878068],
       [0.3736391 , 0.29208687, 0.27337313, 0.05399883, 0.00690219],
       [0.24599251, 0.23275527, 0.3541326 , 0.11754657, 0.04957311],
       [0.27744174, 0.22426778, 0.34556213, 0.11067207, 0.04205634],
       [0.22195794, 0.26750764, 0.37343043, 0.10421793, 0.03288611],
       [0.22334674, 0.27952796, 0.36065865, 0.1186585 , 0.01780811],
       [0.5229349 , 0.22502452, 0.19509155, 0.04250733, 0.01444168],
       [0.52591366, 0.22199042, 0.18906343, 0.05170693, 0.01132548],
       [0.44623193, 0.26057857, 0.21645962, 0.0500822 , 0.02664771],
       [0.41215885, 0.26224953, 0.24817753, 0.05724855, 0.02016549],
       [0.38825643, 0.26196513, 0.2804589 , 0.0544098 , 0.01490974],
       [0.41073698, 0.29060766, 0.23125394, 0.05534397, 0.01205751],
       [0.2526619 , 0.24590246, 0.